# getting started
If you have mamba or conda installed you can use the cell below to create an environment that can be used to run this jupyter notebook in jupyter or vscode
```python
mamba env create --file './nostr.yml'
mamba activate nostr
```
or
```python
conda env create --file './nostr.yml'
conda activate nostr
```

# python-nostr examples
this notebook serves as a set of interactive examples to highlight how nostr and python-nostr work. the cell below will make sure this library is in your python path

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path('../').resolve()))
print(sys.path)

['/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr/example_nbs', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python310.zip', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10/lib-dynload', '', '/Users/ryanarmstrong/mambaforge/envs/nostr/lib/python3.10/site-packages', '/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr']



## key management
python-nostr has basic key management capabilities to generate new private keys or load existing keys.
The base key class is instantiated with raw_bytes - if you are loading an existing hex key you can use `Key.raw_bytes_from_hex()`

The `PublicKey` and `PrivateKey` classes inherit these capabilities. The bech32 representation of the private and public key (i.e. npub... and nsec...) are defined in each class respectively.

**change log from original repo:**
 - Added base Key class to manage shared methods
 - Added hex to key method
 - Changed `raw_secret` to `raw_bytes` in `PrivateKey` to keep classes consistent
    - this change helps the code, but does it add some risk to users?

In [2]:
from pathlib import Path
import secrets
from nostr.key import Key, PrivateKey, PublicKey

base_key = Key(secrets.token_bytes(32))
print(f'this is the raw bytes of a key: {base_key.raw_bytes}\n'
      f'this is the hex of the same key: {base_key.hex()}')

assert base_key.hex() == Key.raw_bytes_from_hex(base_key.hex()).hex()
assert issubclass(PrivateKey, Key) and issubclass(PublicKey, Key)

try:
      base_key.bech32()
except ValueError as e:
      print(e)

this is the raw bytes of a key: b"#B\x1f\xd1\xbb\xaa\xaf\x05\xd5\xdb\xa1\x13\xb5\xb04\x93\x06F:\xa6\xd2\xe6\xb9'i\x96\x87\xecg\r\xa0 "
this is the hex of the same key: 23421fd1bbaaaf05d5dba113b5b0349306463aa6d2e6b927699687ec670da020
bech32 prefix not defined for type <class 'nostr.key.Key'>


Below we use the `PrivateKey` class to make a private key or load it from a file on disk. We can also confirm that the private and public key representations start with the appropriate characters (npub and nsec)

In [3]:
key_file = Path('./private_key.txt') # saving a private key for testing

if not key_file.exists():
    private_key = PrivateKey()
    public_key = private_key.public_key
    print(f"Generated new private key")
    print(f"Public key: {public_key.bech32()}")
    with open(key_file, 'w') as f:
        f.write(private_key.raw_bytes.hex())
else:
    with open(key_file, 'r') as f:
        hex = f.read()
        # private_key = PrivateKey(bytes.fromhex(hex))
        private_key = PrivateKey.raw_bytes_from_hex(hex)
    public_key = private_key.public_key
    print(f'Loaded private key from {key_file}')
    print(f'Public key: {public_key.bech32()}')

assert private_key.bech32()[:4]=='nsec' and public_key.bech32()[:4]=='npub'

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


Next let's instantiate a `RelayManager` and add a couple relay urls. We are going to set one to not write and see what happens

In [4]:
from nostr.relay_manager import RelayManager, Relay

relay_manager = RelayManager()
relay_manager.add_relay('wss://nostr-2.zebedee.cloud')
relay_manager.add_relay('wss://nostr.zebedee.cloud')
relay_manager.add_relay('wss://nostr-relay.lnmarkets.com', write=False)

the relay manager keeps all of the `Relay` objects in a dict keyed by the url. Let's look at some of the properties of the relays we added before we actually do anything

In [7]:
for url, relay in relay_manager.relays.items():
    print(relay, '\n\n')
    assert type(relay) == Relay and url == relay.url


{
  "url": "wss://nostr-2.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr-relay.lnmarkets.com",
  "policy": {
    "read": true,
    "write": false
  },
  "subscriptions": []
} 




Now we can connect:

**changelog from original repo:**
 - added a connection context for easier operations
    - is this actually helpful in context of nostr?


In [8]:
import ssl
from nostr.relay_manager import Connection


with relay_manager.connection({"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
    print('test')

connection open to wss://nostr-2.zebedee.cloud
connection open to wss://nostr.zebedee.cloud
connection open to wss://nostr-relay.lnmarkets.com
test
connection to wss://nostr-2.zebedee.cloud closed
connection to wss://nostr.zebedee.cloud closed
connection to wss://nostr-relay.lnmarkets.com closed


In [19]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType


filters = Filters([Filter(authors=['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'],
                          kinds=[EventKind.TEXT_NOTE],
                          since=int(time.time()-3400)
                          )])
subscription_id = 'recent2'
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())

print(request)

['REQ', 'recent2', {'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720'], 'since': 1671929381}]


In [25]:
with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  time.sleep(1.25) # allow the connections to open
  message = json.dumps(request)
  print(message)
  relay_manager.publish_message(message)
  relay_manager.add_subscription(subscription_id, filters)
  time.sleep(1) # allow the messages to send
  while relay_manager.message_pool.has_events():
    event_msg = relay_manager.message_pool.get_event()
    print(event_msg.event.to_json_object())
  while relay_manager.message_pool.has_notices():
    notice_msg = relay_manager.message_pool.get_notice()
    print(notice_msg.event.to_json_object())
  while relay_manager.message_pool.has_eose_notices():
    eose_notice_msg = relay_manager.message_pool.get_eose_notice()
    print(eose_notice_msg.subscription_id)

# with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification


connection open to wss://nostr-2.zebedee.cloud
connection open to wss://nostr.zebedee.cloud
connection open to wss://nostr-relay.lnmarkets.com
["REQ", "recent2", {"kinds": [1], "authors": ["c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720"], "since": 1671929381}]
{'id': '027e663627ae159589f2fbea066a326827d487970ea05d2d8bdbe5d680408dd7', 'pubkey': 'c80b5248fbe8f392bc3ba45091fb4e6e2b5872387601bf90f53992366b30d720', 'created_at': 1671929565, 'kind': 1, 'tags': [['e', 'f706b1abd879eb9faa601cdc8ee8b6cbca01939e00c7498e717a625037484ee6'], ['e', '9bedbbf60736374dcd87cd3ad3243198695dc199f6f4bc0435b75ec9aa2097a8'], ['p', '5fd693e61a7969ecf5c11dbf5ce20aedac1cea71721755b037955994bf6061bb'], ['p', 'ec8f72ff2937c197cb0d032dae27bae073ae6a4e1bd2a8e2ef1578636b3595cb'], ['p', 'dedf91f5c5eee3f3864eec34b28fc99c6a8cc44b250888ccf4d0d8d854f48d54'], ['p', '32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245'], ['p', '2f02d76f09666ae076e65beb60ff195eb7f44b51c73147a6a37748bfabd60a7c

In [21]:
for url, relay in relay_manager.relays.items():
    print(f'{relay.url=}')
    print(f'\t{relay.policy.should_read=}')
    print(f'\t{relay.policy.should_write=}')
    print(f'\t{relay.subscriptions=}')
    assert type(relay) == Relay and url == relay.url


relay.url='wss://nostr-2.zebedee.cloud'
	relay.policy.should_read=True
	relay.policy.should_write=True
	relay.subscriptions={'recent2': <nostr.subscription.Subscription object at 0x1055f4190>}
relay.url='wss://nostr.zebedee.cloud'
	relay.policy.should_read=True
	relay.policy.should_write=True
	relay.subscriptions={'recent2': <nostr.subscription.Subscription object at 0x1055f4190>}
relay.url='wss://nostr-relay.lnmarkets.com'
	relay.policy.should_read=True
	relay.policy.should_write=True
	relay.subscriptions={'recent2': <nostr.subscription.Subscription object at 0x1055f4190>}


In [22]:
relay_manager.message_pool._unique_events=set()

In [23]:
relay_manager.

SyntaxError: invalid syntax (3692258011.py, line 1)

In [24]:
relay = relay_manager.relays['wss://nostr-2.zebedee.cloud']

In [18]:
relay.ws.

In [39]:
request = [ClientMessageType.CLOSE, subscription_id]

print(request)

with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  time.sleep(1.25) # allow the connections to open
  message = json.dumps(request)
  relay_manager.publish_message(message)
  relay_manager.close_subscription(subscription_id)


['CLOSE', 'recent']
connection open to wss://nostr.zebedee.cloud
connection open to wss://nostr-relay.lnmarkets.com
connection to wss://nostr.zebedee.cloud closed
connection to wss://nostr-relay.lnmarkets.com closed


KeyError: 'recent'

In [20]:
relay_manager.message_pool.notices.empty()

True

In [13]:
test = filters[0]
test.to_json_object()

{'kinds': [<EventKind.TEXT_NOTE: 1>],
 'authors': ['82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'],
 'limit': 1}

In [14]:
request

['CLOSE', '?']

In [23]:
with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}):
    relay_manager.message_pool.events.get()

KeyboardInterrupt: 

In [ ]:
relay_manager

In [18]:
relay_manager.message_pool.notices.empty()

True

In [13]:
public_key.raw_bytes

b'X\x83\x92\xc0]\x91=!vc\xc5\x9d:xW\xd4a\x03\xdfz\n&\xf6\xc8\xd3-L\xfb2R0\xb9'

In [12]:
from nostr import bech32

bech32.bech32_decode(public_key.bech32())[1]

<Encoding.BECH32: 1>

In [32]:
import json
import ssl
import time
from nostr.relay_manager import RelayManager

relay_manager = RelayManager()
relay_manager.add_relay("wss://nostr.zebedee.cloud")
relay_manager.add_relay("wss://nostr-relay.lnmarkets.com")
relay_manager.open_connections({"cert_reqs": ssl.CERT_NONE}) # NOTE: This disables ssl certificate verification
time.sleep(1.25) # allow the connections to open

while relay_manager.message_pool.has_notices():
  notice_msg = relay_manager.message_pool.get_notice()
  print(notice_msg.content)
  
relay_manager.close_connections()

connection open to wss://nostr.zebedee.cloud
connection open to wss://nostr-relay.lnmarkets.com
connection to wss://nostr.zebedee.cloud closed
connection to wss://nostr-relay.lnmarkets.com closed


In [33]:
relay_manager.relays

{'wss://nostr.zebedee.cloud': <nostr.relay.Relay at 0x109f06b00>,
 'wss://nostr-relay.lnmarkets.com': <nostr.relay.Relay at 0x109f06fe0>}

In [43]:
import json 
import ssl
import time
from nostr.event import Event
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType
from nostr.key import PrivateKey

relay_manager.open_connections({"cert_reqs": ssl.CERT_NONE}) # NOTE: This disables ssl certificate verification
time.sleep(2) # allow the connections to open

event = Event(private_key.public_key.hex(), "Hello Nostr - testing from python-nostr")
event.sign(private_key.hex())

message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
relay_manager.publish_message(message)
time.sleep(1) # allow the messages to send

relay_manager.close_connections()

connection open to wss://nostr-2.zebedee.cloud
connection open to wss://nostr.zebedee.cloud
connection open to wss://nostr-relay.lnmarkets.com
connection to wss://nostr-2.zebedee.cloud closed
connection to wss://nostr.zebedee.cloud closed
connection to wss://nostr-relay.lnmarkets.com closed


In [16]:
Event()

[]

In [ ]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType

filters = Filters([Filter(authors=[private_key.public_key.hex()], kinds=[EventKind.TEXT_NOTE])])
subscription_id = <a string to identify a subscription>
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())

relay_manager = RelayManager()
relay_manager.add_relay("wss://nostr-pub.wellorder.net")
relay_manager.add_relay("wss://relay.damus.io")
relay_manager.add_subscription(subscription_id, filters)
relay_manager.open_connections({"cert_reqs": ssl.CERT_NONE}) # NOTE: This disables ssl certificate verification
time.sleep(1.25) # allow the connections to open

message = json.dumps(request)
relay_manager.publish_message(message)
time.sleep(1) # allow the messages to send

while relay_manager.message_pool.has_events():
  event_msg = relay_manager.message_pool.get_event()
  print(event_msg.event.content)
  
relay_manager.close_connections()